In [1]:
import torch
from torch import nn, tensor, optim
import numpy as np
from lib.activations import ReLU, Softmax
from lib.dataset import X, y
from lib.losses import CrossEntropyLoss
from lib.network import Sequential, Linear
from lib.network_base import Module
from lib.utilities import one_hot_encoding

In [2]:
a = np.array(
    [
        [6.7776, -3.6296],
        [-4.08, 4.42],
        [-4.08, 4.42],
        [6.1599, -4.1828],
    ]
)
preds = Softmax()(a)
Y_onehot = one_hot_encoding(y, 2)

In [5]:
np.testing.assert_allclose(
        preds,
        Y_onehot,
        atol=1e-3,
        err_msg=f"The model predicts the wrong classes. Ground-truth: {Y_onehot}, predictions: {preds}",
    )

In [3]:
preds - Y_onehot

array([[-3.02132432e-05,  3.02132432e-05],
       [ 2.03426978e-04, -2.03426978e-04],
       [ 2.03426978e-04, -2.03426978e-04],
       [-3.22261531e-05,  3.22261531e-05]])

In [2]:
w1 = np.array(
    [
        [3.21,-2.34, 3.16],
        [3.21,-2.34, 3.16],
    ]
)
b1 = np.array([-3.21, 2.34, -3.16])

w2 = np.array(
    [
        [0.03, -5.84],
        [4.64, -3.44],
        [3.21, 3.21],
    ]
)
b2 = np.array([-4.08,4.42])


In [3]:
y = np.array(
    [
        [6.7776, -3.6296],
        [-4.08, 4.42],
        [-4.08, 4.42],
        [6.1599, -4.1828],
    ]
)

In [4]:
# w2 = np.array(
#     [
#         [(10.2399/3.21)-3.16, -(8.6028/3.21)-3.16],
#         [10.8576/2.34, -8.0496/2.34],
#         [3.21, 3.21],
#     ]
# )

In [5]:
np.maximum((X @ w1) + b1,0) @ w2 +b2

array([[ 6.7776, -3.6296],
       [-4.08  ,  4.42  ],
       [-4.08  ,  4.42  ],
       [ 6.1599, -4.1828]])

In [6]:
y

array([[ 6.7776, -3.6296],
       [-4.08  ,  4.42  ],
       [-4.08  ,  4.42  ],
       [ 6.1599, -4.1828]])

In [17]:
np.array([
    [1,0,0],
    [0,0,1],
    [0,0,1],
    [0,1,0],
]) @ np.array([
    [2.6976,0.7904],
    [2.0799,  0.2372],
    [-8.16, 8.84],
])

array([[ 2.6976,  0.7904],
       [-8.16  ,  8.84  ],
       [-8.16  ,  8.84  ],
       [ 2.0799,  0.2372]])

In [ ]:
np.array([[ 2.6976,  0.7904],
       [-8.16  ,  8.84  ],
       [-8.16  ,  8.84  ],
       [ 2.0799,  0.2372]])

In [3]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.layers = [
            Linear(2, 2),
            ReLU(),
            Linear(2, 1),
        ]
        # weight 1
        self.layers[0].W.data = np.array(
            [
                [3.21, -2.34],
                [3.21, -2.34],
            ]
        )
        # bias 1
        self.layers[0].b.data = np.array([-3.21, 2.34])
        # weight 2
        self.layers[2].W.data = np.array(
            [
                [3.19, -2.68],
                [4.64, -3.44],
            ]
        )
        # bias 2
        self.layers[2].b.data = np.array([-4.08, 4.42])
    def forward(self, X):
        for i, layer in enumerate(self.layers):
            X = layer(X)
            print(i, X)
        return X

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleList(
            [
                nn.Linear(2, 2),
                nn.ReLU(),
                nn.Linear(2, 1),
            ]
        )
        self.layers[0].weight = nn.Parameter(
            tensor(
                [
                    [3.21, -2.34],
                    [3.21, -2.34],
                ]
            ).T,
            requires_grad=True,
        )
        self.layers[0].bias = nn.Parameter(tensor([-3.21, 2.34]), requires_grad=True)

        self.layers[2].weight = nn.Parameter(
            tensor(
                [
                    [3.19, -2.68],
                    [4.64, -3.44],
                ]
            ).T,
            requires_grad=True,
        )
        self.layers[2].bias = nn.Parameter(tensor([-4.08, 4.42]), requires_grad=True)
    def forward(self, X):
        for i, layer in enumerate(self.layers):
            X = layer(X)
            print(i, X)
        return X

In [5]:
net = Net()
net2 = Net2()

In [10]:
X

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [6]:
for name, param in net.named_parameters():
    print(name, param)

layers.0.weight Parameter containing:
tensor([[ 3.2100,  3.2100],
        [-2.3400, -2.3400]], requires_grad=True)
layers.0.bias Parameter containing:
tensor([-3.2100,  2.3400], requires_grad=True)
layers.2.weight Parameter containing:
tensor([[ 3.1900,  4.6400],
        [-2.6800, -3.4400]], requires_grad=True)
layers.2.bias Parameter containing:
tensor([-4.0800,  4.4200], requires_grad=True)


In [7]:
X_ = tensor(X,dtype=torch.float32)

In [8]:
net(X_)

0 tensor([[-3.2100,  2.3400],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 3.2100, -2.3400]], grad_fn=<AddmmBackward0>)
1 tensor([[0.0000, 2.3400],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [3.2100, 0.0000]], grad_fn=<ReluBackward0>)
2 tensor([[ 6.7776, -3.6296],
        [-4.0800,  4.4200],
        [-4.0800,  4.4200],
        [ 6.1599, -4.1828]], grad_fn=<AddmmBackward0>)


tensor([[ 6.7776, -3.6296],
        [-4.0800,  4.4200],
        [-4.0800,  4.4200],
        [ 6.1599, -4.1828]], grad_fn=<AddmmBackward0>)

In [9]:
net2(X)

0 [[-3.21  2.34]
 [ 0.    0.  ]
 [ 0.    0.  ]
 [ 3.21 -2.34]]
1 [[0.   2.34]
 [0.   0.  ]
 [0.   0.  ]
 [3.21 0.  ]]
2 [[ 6.7776 -3.6296]
 [-4.08    4.42  ]
 [-4.08    4.42  ]
 [ 6.1599 -4.1828]]


array([[ 6.7776, -3.6296],
       [-4.08  ,  4.42  ],
       [-4.08  ,  4.42  ],
       [ 6.1599, -4.1828]])

In [ ]:
layers = nn.ModuleList(
    [
        nn.Linear(3, 2),
        nn.ReLU(),
        nn.Linear(2, 1),
        nn.Sigmoid(),
    ]
)

In [ ]:
layers[1]

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleList(
            [
                nn.Linear(3, 2),
                nn.ReLU(),
                nn.Linear(2, 1),
                nn.Sigmoid(),
            ]
        )
        self.layers[0].weight = nn.Parameter(
            tensor(
                [
                    [-2.0, 2.0, -3.0],
                    [1.0, 0.0, 1.0],
                ]
            ),
            requires_grad=True,
        )
        self.layers[0].bias = nn.Parameter(tensor([3.0, 0.0]), requires_grad=True)

        self.layers[2].weight = nn.Parameter(tensor([[-1.0, 1.0]]), requires_grad=True)
        self.layers[2].bias = nn.Parameter(tensor([-3.0]), requires_grad=True)

    def forward(self, X):
        for i, layer in enumerate(self.layers):
            X = layer(X)
            print(i, X)
        return X

In [ ]:
X = tensor(
    [
        [1.0, 2.0, 3.0],
        [3.0, 4.0, 5.0],
    ]
)
y = tensor([[0.0, 1.0]]).T

In [ ]:
net = Net()
lossfun = nn.BCELoss()

In [ ]:
y_hat = net(X)
loss = lossfun(y_hat, y)